# **Script Python - Antoine Petiteau et Luc Le Mée - Brain Tumor Radiogenomic Classification**

## Liste des librairies utilisées

Une partie du code se base sur le travail suivant : https://www.kaggle.com/chumajin/brain-tumor-eda-for-starter-english-version

In [2]:
import numpy as np 
import pandas as pd 
import os
import pydicom
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
import seaborn as sns
import glob
from PIL import Image
import random
import cv2


#Pour les réseaux de neuronnes :
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.layers import Dropout

## Etude des fichiers à disposition

In [2]:
sample = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv")
print(len(sample)) #87 individus dans l'échantillon test
sample.head(10) #Visualise les 10 premières lignes

Commentaire : BraTS21ID : Correspond à l'identifiant. Permet de faire le lien avec les images
MGMT_value : proba de la présence de MGMT. 1 = Présence. Détecter le MGMT permet de prévenir la tumeur du cerveau

## Base train

In [3]:
train = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
print(len(train))#585 individus dans le fichier train
train.head(10)#Visualise les 10 premières lignes

In [4]:
sns.countplot(data=train, x='MGMT_value')

Il y a légèrement plus d'individus ayant la présence de MGMT

## Visualisation d'une image

In [8]:
dataset = pydicom.filereader.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00005/FLAIR/Image-109.dcm') #Importation d'une seule image pour visualiser
img = dataset.pixel_array #La fonction permet de lire les pixel dans une forme plus pratique
fig, ax = plt.subplots()#Création d'un plot
#ax.imshow(img, cmap='Greys')#Charge l'image en nuance de gris
ax.imshow(img, cmap = "Greys")
ax.set_axis_off()# Pas de visualisation des axes car il s'agit d'une image
plt.show() #Affiche l'image

In [6]:
height = np.size(img, 0)
width = np.size(img, 1)
print("Taille de l'image : hauteur = " , height, ", largeur = ", width)

Les images sont de taille 512x512.
D'après les recherches, il est pertinent de diminuer la taille et de passer en 256x256. 

## Etude des images

In [7]:
train["imfolder"] = ['{0:05d}'.format(s) for s in train["BraTS21ID"]]
#0:05d --> Permet d'ajouter quatre "0"
#L'objectif de cette commande est de relier chaque identifiant à un identifiant de fichier qui contient les images associées à l'identifiant
train.head(10)

In [8]:
#On définit le chemin d'accès aux images associées à chaque id
train_path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train" #base du chemin, commune à tous les id
train["path"] = [os.path.join(train_path,s) for s in train["imfolder"]  ]# Ajout du numéro de dossier définit précédement
train.head(10)#On visualise la colonne qui contient le chemin vers les images

In [9]:
#Défini une liste :
Scans = ["FLAIR","T1w","T1wCE","T2w"]
#Utile pour la suite, pour la navigation au sein des fichiers avec une boucle

Calcul du nombre d'images par dossier pour le premier id

In [10]:
for type_img in Scans :
    print(type_img , " : ", len(os.listdir(os.path.join(train["path"].iloc[1],type_img))), " images")


In [11]:
for scan in Scans: #Compte le nombre de sous-dossiers parmi les dossiers définit dans "Scans" 
    train[scan +"_count"] = [ len(os.listdir(os.path.join(train["path"].iloc[s],scan))) for s in tqdm(range(len(train))) ]#Ajout de 4 colonnes

Il y a bien 585 dossiers.
Tout est complet pour les 585 individus

In [12]:
train.head(10)

Il n'y a pas forcément le même nombre d'images dans tous les sous-dossiers en fonction des individus.
Dépend du type de plan (coronal, axial et sagital)
Dans tous les sous dossiers il y a une majorité de plan de type "axial"

In [13]:
allsame = [train["FLAIR_count"].iloc[s] ==   train["T1w_count"].iloc[s] ==train["T1wCE_count"].iloc[s] ==train["T2w_count"].iloc[s] 
          for s in range(len(train))]

Le programme ci-dessus permet de déterminer s'il y a le même nombre d'image dans les 4 coupes pour chaque individu.
Création d'une variable pour stocker cette information

In [14]:
train["allsame"] = allsame #Création de la variable 
train.head(10)

In [15]:
round(train["allsame"].sum()/len(train) * 100,2)

Commentaire : 10.77 % des individus ont le même nombre d'image dans chaque type de coupe

## EDA : Exemple 

In [16]:
row_ID = 64
train["BraTS21ID"].iloc[row_ID] #A la ligne 64 on retrouve l'individu ayant le numéro id 100

On prend l'exemple de l'individu 100 car il y a la présence de MGMT et il y a le même nombre d'image pour chaque coupe. Donc plus simple à traiter

In [17]:
temp_folder = train["path"].iloc[row_ID]
temp_folder

In [18]:
temp_folder2 = os.path.join(temp_folder,"FLAIR")
temp_files = os.listdir(temp_folder2)
temp_files[:3]

Donne les 3 permières images de l'individu 100 dans 'FLAIR'

In [19]:
imagenum = [s.split("-")[1] for s in temp_files]
imagenum = [s.split(".")[0] for s in imagenum]
imagenum[:3]

Permet d'extraire le numéro de l'image.
La première fonction split() récupère ce qui se trouve après le caractère "-".
La deuxième fonction split() récupère ce qui se trouve avant le caractère ".".
Donc on récupère le numéro.

In [20]:
temp_path = [os.path.join(temp_folder2,s) for s in temp_files]
temp_path[:3]

Permet d'obtenir les chemins d'accès pour toutes les images spécifiées précédement

In [21]:
tempdf = pd.DataFrame()
tempdf["image_num"] = imagenum  #Défini précédement
tempdf["image_num"] = tempdf["image_num"].astype("int") #Converti en numérique

tempdf["temp_path"] = temp_path
tempdf.head(3)

In [22]:
tempdf = tempdf.sort_values("image_num").reset_index(drop=True)
tempdf.head(3)

Permet de remettre en ordre les données.

In [23]:
#Sauvegarde
finpath = tempdf["temp_path"]

## Application pour toutes les données :

In [24]:
def makepath(row_ID,scan):
    
    temp_folder = train["path"].iloc[row_ID]
    temp_folder2 = os.path.join(temp_folder,scan)#Pour joindre 2 éléments sur un même chemin d'accès
    temp_files = os.listdir(temp_folder2) #Donne la liste des documents présent dans le dossier "temp_folder2"
    imagenum = [s.split("-")[1] for s in temp_files]
    imagenum = [s.split(".")[0] for s in imagenum]
    temp_path = [os.path.join(temp_folder2,s) for s in temp_files] # temp_files correspond à la liste des images dans le dossier
    tempdf = pd.DataFrame()
    tempdf["image_num"] = imagenum
    tempdf["image_num"] = tempdf["image_num"].astype("int")
    tempdf["temp_path"] = temp_path
    tempdf = tempdf.sort_values("image_num").reset_index(drop=True) #Remet à zéro l'index
    finpath = tempdf["temp_path"]
    return finpath

In [25]:
row_id=64

sampledf = pd.DataFrame()
for scan in Scans:
    sampledf[scan + "_path"] = makepath(row_id,scan) #Utilise la fonction makepath définie précédement
sampledf.head(5)

Donne l'accès aux images pour chaque type de coupe.

## Visualisation

Fonction pour lire les images DICOM :

In [26]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    data = cv2.resize(data, dsize=(256, 256), interpolation=cv2.INTER_CUBIC) #Permet de réduire la taille de l'image

    return data

In [27]:
#Autre fonction :
def makeimg(path):
    dataset = pydicom.filereader.dcmread(path)#Pour lire un fichier DICOM (image)
    img = dataset.pixel_array #Mise en forme de l'image à partir des pixel
    img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC) #Permet de réduire la taille de l'image

    return img

In [28]:
load_dicom(sampledf["FLAIR_path"].iloc[0]).shape

In [29]:
plt.figure(figsize=(80,10)) #Création plot
for num,scan in enumerate(Scans): #enumerate permet d'obtenir à la fois le numéro d'itération de la boucle et sa valeur ("FLAIR","T1w","T1wCE" ou"T2w")
        img = load_dicom(sampledf[scan + "_path"].iloc[16]) #Fonction makeimg définie précédement pour lire les fichiers DICOM
        plt.subplot(4,25,num+1)#Permet de créer plusieurs plot sur une même fenêtre graphique
        #plt.axis("off")
        plt.imshow(img) #Affiche les data comme étant des images
        
        plt.title(scan,fontsize=18) #Titre seulement sur la première ligne
      

In [30]:
print("MGMT_value = " + str(train["MGMT_value"].iloc[row_id])) #Affiche "MGMT_value = 1" car c'est le cas pour l'individu 64


for row in range(len(sampledf)): #Pour chaque ligne, donc chaque image
    plt.figure(figsize=(80,10)) #Création plot
    for num,scan in enumerate(Scans): #enumerate permet d'obtenir à la fois le numéro d'itération de la boucle et sa valeur ("FLAIR","T1w","T1wCE" ou"T2w")
        img = load_dicom(sampledf[scan + "_path"].iloc[row]) #Fonction makeimg définie précédement pour lire les fichiers DICOM
        plt.subplot(4,25,num+1)#Permet de créer plusieurs plot sur une même fenêtre graphique
        #plt.axis("off")
        plt.imshow(img) #Affiche les data comme étant des images
        
        if row==0:
            plt.title(scan,fontsize=18) #Titre seulement sur la première ligne
        if num==0:
            plt.ylabel("row=" + str(row),fontsize=18) #Numéro de la ligne seulement sur la gauche de la première image

En haut à droite de plusieurs images il y a un élément qui ressort. On suppose qu'il s'agit de la présence de MGMT.

On visualise la même chose pour un individu ayant MGMT_value = 0 :

In [31]:
row_id = 65 #MGMT_value = 0 pour cet individu

sampledf = pd.DataFrame()
for scan in Scans:
    sampledf[scan + "_path"] = makepath(row_id,scan)

print("MGMT_value = " + str(train["MGMT_value"].iloc[row_id]))

plt.figure(figsize=(80,10)) #Création plot
for num,scan in enumerate(Scans): #enumerate permet d'obtenir à la fois le numéro d'itération de la boucle et sa valeur ("FLAIR","T1w","T1wCE" ou"T2w")
        img = load_dicom(sampledf[scan + "_path"].iloc[16]) #Fonction makeimg définie précédement pour lire les fichiers DICOM
        plt.subplot(4,25,num+1)#Permet de créer plusieurs plot sur une même fenêtre graphique
        #plt.axis("off")
        plt.imshow(img) #Affiche les data comme étant des images
        
        plt.title(scan,fontsize=18) #Titre seulement sur la première ligne
    

In [32]:
for row in range(len(sampledf)):
    plt.figure(figsize=(80,10))
    for num,scan in enumerate(Scans):
        img = load_dicom(sampledf[scan + "_path"].iloc[row])
        plt.subplot(4,25,num+1)
        #plt.axis("off")
        plt.imshow(img)
        
        if row==0:
            plt.title(scan,fontsize=18)
        if num==0:
            plt.ylabel("row=" + str(row),fontsize=18)

Pour cet individu il n'y a pas la marque en haut à droite comme on avait pu le constater sur le cas précédent.

Les premières et les dernières images sont toutes noires. Si on prend un échantillon il vaut mieux prendre les images du milieu.

On vient d'étudier le cas ou toutes les coupes ont le même nombre d'images. Cependant, pour 90 % de la base de données, il n'y a pas le même nombre d'images dans chaque coupe.
Il faut étudier ces cas là. On regarde avec l'ID 0.

In [33]:
train.head(1) #Individu ayant l'ID = 0

On voit qu'il n'y a pas le même nombre d'image dans chaque coupe.
On s'interesse d'abbord à la coupe T1w car c'est celle qui contient le moins d'images.

In [34]:
row_id = 0 #Individu 0

sampledf = pd.DataFrame()
for scan in Scans:
    sampledf[scan + "_path"] = makepath(row_id,scan)
sampledf #Chemin d'accès vers les images

In [35]:
sampledf["T1w_path"].iloc[32] #Dernière images car l'index commence à 0

In [36]:
print("MGMT_value = " + str(train["MGMT_value"].iloc[row_id]))

for row in range(10):
    plt.figure(figsize=(80,5))
    for num,scan in enumerate(Scans):
        img = load_dicom(sampledf[scan + "_path"].iloc[row])
        plt.subplot(4,33,num+1)
        #plt.axis("off")
        plt.imshow(img)
        
        if row==0:
            plt.title(scan,fontsize=18)
        if num==0:
            plt.ylabel("row=" + str(row),fontsize=18)

FLAIR et T2w restent sombres. Ce sont eux qui ont le plus d'images et il semble que l'image n'apparaisse pas sur les 33 premiers fichiers mais probablement plustard.

## Vérification sur la base test : combien d'individus ont le même nombre d'images pour toutes les coupes ?

Reprend les mêmes éléments de code que précédement mais appliqués à la base test

In [37]:
sample["imfolder"] = ['{0:05d}'.format(s) for s in sample["BraTS21ID"]]

test_path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"

sample["path"] = [os.path.join(test_path,s) for s in sample["imfolder"]  ]


allres = []

for scan in Scans:
    sample[scan +"_count"] = [ len(os.listdir(os.path.join(sample["path"].iloc[s],scan))) for s in tqdm(range(len(sample))) ]


In [38]:
sample.head(5) #Les données du nombre d'images par couche sont récupées

In [39]:
allsame = [sample["FLAIR_count"].iloc[s] ==   sample["T1w_count"].iloc[s] ==sample["T1wCE_count"].iloc[s] ==sample["T2w_count"].iloc[s] 
          for s in range(len(sample))]

sample["allsame"] = allsame

sample.head(5)#La variable allsame permet de voir s'il y a le même nombre d'image dans chaque couche

In [40]:
sample["allsame"].sum()/len(sample) * 100

Pour 12,64 % de l'échantillon test il y a le même nombre d'images dans chaque couche

## Test pour la soumission

In [41]:
train.groupby("allsame")["MGMT_value"].mean().reset_index()

Permet de voir la proportion d'individu à avoir la présence de MGMT en fonction de l'égalité, ou non, du nombre d'image par coupe.
On voit qu'il y a une légère différence ce qui peut nous permettre de tester ces valeurs comme des prévisions. Aucun interet statistique mais permet de faire un test et de se familiariser avec le submit.

In [42]:
sample["MGMT_value"] = np.where(sample["allsame"],0.460317,0.532567) #Si la variable allsame=1 alors on affecte la proba 0.460317 à la variable MGMT_value. sinon 0.532567
sample.head(5)

In [43]:
sample= sample[["BraTS21ID","MGMT_value"]] #On ne conserve que ces deux colonnes

In [44]:
#sample.to_csv("submission.csv",index=False) #Sauvegarde en csv
sample.head(10)

# Préparation à la modélisation

## Echantillon train

In [45]:
len(train)

La base train comporte 585 individus.
On retire 3 individus qui sont à exclure d'après l'organisation de la compétition.

In [46]:
train = train[~train.BraTS21ID.isin([109,123,709])] #Retire les 3 individus
len(train)

Les 3 individus ont bien été retirés. On sélectionne un échantillon de 400 individus.

In [47]:
random.seed(1234)
list_ind = random.sample(list(train.BraTS21ID), 500)#Sélectionne 30 ID au hasard
train2 = train[train.BraTS21ID.isin(list_ind)]#Création de l'échantillon avec les 30 individus
len(train2)

On obtient bien un echantillon de 400 individus. On vérifie la répartition :

In [48]:
sns.countplot(data=train2, x='MGMT_value')

In [49]:
train2.head(5)

Il y a beaucoup d'images. On va récupérer l'image du milieu pour chaque individu.
Problème : les numéros d'image pour chaque coupe ne commencent pas forcément à 1.

Fonction pour trouver l'image centrale de chaque type de coupe, pour chaque individu :

In [50]:
def img_centrale(path,scan):
    temp_folder = os.path.join(path,scan)#Chemin d'accès d'un individu à une coupe précise (scan)
    temp_files = os.listdir(temp_folder)#Permet d'obtenir la liste des fichiers (images) présents dans le dossier
    
    #Pour isoler le numéro d'image :
    imagenum = [s.split("-")[1] for s in temp_files]
    imagenum = [s.split(".")[0] for s in imagenum]

    tempdf = pd.DataFrame()
    tempdf["image_num"] = imagenum  #Défini précédement
    tempdf["image_num"] = tempdf["image_num"].astype("int") #Converti en numérique

    tempdf = tempdf.sort_values("image_num")#Tri sur la liste d'image
    image_centrale = tempdf["image_num"].iloc[int(len(tempdf)/2)]#Retourne le numéro de l'image au milieu de la liste
    
    return image_centrale

Test sur le premier individu et le type de coupe "FLAIR" : 

In [51]:
print(img_centrale(train2["path"].iloc[0],"FLAIR"))

Pour chaque individu, on stock le lien de l'image centrale dans une colonne :

In [52]:
for scan in Scans : #Pour chaque type de coupe
    train2["path_img_" + scan] = [os.path.join(train2["path"].iloc[i],scan,"Image-"+ str(img_centrale(train2["path"].iloc[i],scan)) +".dcm") for i in range(len(train2))]

In [53]:
print(train2["path_img_T1wCE"].iloc[5])#Pour le 6ème individu
train2.head(5)

On supprime les 4 colonnes qui contiennent le nombre d'image car cette information n'est plus utile

In [54]:
del train2["FLAIR_count"]
del train2["T1w_count"]
del train2["T1wCE_count"]
del train2["T2w_count"]

Observation d'une image pour le premier individu :

In [55]:
img = load_dicom(train2.path_img_FLAIR.iloc[0])#La fonction makeimg a été définit avant dans le script
plt.imshow(img)

En observant les 4 coupes :

In [56]:
plt.figure(figsize=(15,5)) #Tailles des images sur l'écran

for num,scan in enumerate(Scans): #Pour chaque type de coupe
        
        img = load_dicom(train2["path_img_" +scan].iloc[0]) #iloc[0] donc premier individu
        plt.subplot(1,4,num+1)#Sépare la fenêtre graphique en 4 
        plt.imshow(img)#Fait afficher l'image
        plt.xticks([])#Pas de xlab
        plt.yticks([])#Pas de ylab
        plt.title(scan,fontsize=18) #Titre au dessus des images

print("Taille des images : " + str(img.shape))

On obtient bien chaque type de coupe pour le premier individu de l'échantillon. La taille des images a été réduite à 256x256.

 Stocker les images :

In [57]:
#Déclaration des listes d'images
train2_FLAIR = []
train2_T1w = []
train2_T1wCE = []
train2_T2w = []

#Alimentation des listes d'images
#FLAIR
for i in range(len(train2)) :
    train2_FLAIR.append(load_dicom(train2["path_img_FLAIR"].iloc[i]))
#T1w
for i in range(len(train2)) :
    train2_T1w.append(load_dicom(train2["path_img_T1w"].iloc[i]))
#T1wCE
for i in range(len(train2)) :
    train2_T1wCE.append(load_dicom(train2["path_img_T1wCE"].iloc[i]))
#T2w
for i in range(len(train2)) :
    train2_T2w.append(load_dicom(train2["path_img_T2w"].iloc[i]))
    
train2_FLAIR = np.array(train2_FLAIR)

#Normaliser les images :
train2_FLAIR = train2_FLAIR.astype('float32')
train2_FLAIR /= 255

#Reshape : 
img_rows=train2_FLAIR[0].shape[0]
img_cols=train2_FLAIR[0].shape[1]
train2_FLAIR=train2_FLAIR.reshape(train2_FLAIR.shape[0],img_rows,img_cols,1)

MGMT = np.array(train2["MGMT_value"])

print("train2_FLAIR : " + str(type(train2_FLAIR)) + "; MGMT : " + str(type(MGMT) ))
print("train2_FLAIR : " + str(train2_FLAIR.shape) + "; MGMT : " + str(MGMT.shape ))

Il est surement possible de générer une boucle pour remplir les 4 types de coupe en même temps. A creuser si nous avons le temps.

Chaque liste contient les photos associés à la base train2. Nous avons les images pour la modélisation.

Exemple au hasard pour un élément d'une liste :

In [58]:
plt.imshow(train2_T2w[8])

On fait afficher les 30 premiers FLAIR :

In [59]:
plt.figure(figsize=(10,10))#Taille des images
for i in range(30) :#Pour les 30 premiers FLAIR de l'échantillon
    plt.subplot(5,6,i+1)#Divise la fenêtre graphique (30 images)
    plt.imshow(train2_FLAIR[i])
    plt.xticks([])
    plt.yticks([])

Il y a différents types de plan (coronal, axial et sagital).

Idée pour la suite : passer les images en noir et blanc (ou nuance de gris) --> permet d'améliorer le temps d'execution ?

## Création d'un échantillon test :

Tirage aléatoire :

In [60]:
test2 = train[~train.BraTS21ID.isin(list_ind)]#Retire les individus présents dans la base train2
len(test2)

Récupérer l'image centrale :

In [61]:
for scan in Scans : #Pour chaque type de coupe
    test2["path_img_" + scan] = [os.path.join(test2["path"].iloc[i],scan,"Image-"+ str(img_centrale(test2["path"].iloc[i],scan)) +".dcm") for i in range(len(test2))]

Stocker les images :

In [62]:
#Déclaration des listes d'images
test2_FLAIR = []
test2_T1w = []
test2_T1wCE = []
test2_T2w = []

#Alimentation des listes d'images
#FLAIR
for i in range(len(test2)) :
    test2_FLAIR.append(load_dicom(test2["path_img_FLAIR"].iloc[i]))
#T1w
for i in range(len(test2)) :
    test2_T1w.append(load_dicom(test2["path_img_T1w"].iloc[i]))
#T1wCE
for i in range(len(test2)) :
    test2_T1wCE.append(load_dicom(test2["path_img_T1wCE"].iloc[i]))
#T2w
for i in range(len(test2)) :
    test2_T2w.append(load_dicom(test2["path_img_T2w"].iloc[i]))
    
test2_FLAIR = np.array(test2_FLAIR)

#Normaliser les images :
test2_FLAIR = test2_FLAIR.astype('float32')
test2_FLAIR /= 255

#Reshape : 
img_rows=test2_FLAIR[0].shape[0]
img_cols=test2_FLAIR[0].shape[1]
test2_FLAIR=test2_FLAIR.reshape(test2_FLAIR.shape[0],img_rows,img_cols,1)

MGMT_test = np.array(test2["MGMT_value"])

Vérification :

In [63]:
plt.figure(figsize=(10,10))#Taille des images
for i in range(30) :#Pour les 30 premières images FLAIR de l'échantillon
    plt.subplot(5,6,i+1)#Divise la fenêtre graphique (30 images)
    plt.imshow(test2_FLAIR[i])
    plt.xticks([])
    plt.yticks([])

# Formater l'échantillon test de la soumission

On veut récupérer la liste d'image pour pouvoir l'injecter dans notre modèle et faire des prévisions.

In [64]:
test_sub = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv")
print(len(test_sub)) #87 individus dans l'échantillon test
test_sub.head(10) #Visualise les 10 premières lignes

In [65]:
test_sub["imfolder"] = ['{0:05d}'.format(s) for s in test_sub["BraTS21ID"]]
#On définit le chemin d'accès aux images associées à chaque id
test_sub_path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test" #base du chemin, commune à tous les id
test_sub["path"] = [os.path.join(test_sub_path,s) for s in test_sub["imfolder"]  ]# Ajout du numéro de dossier définit précédement
test_sub.head(10)#On visualise la colonne qui contient le chemin vers les images

Recherche de l'image centrale :

In [66]:
for scan in Scans : #Pour chaque type de coupe
    test_sub["path_img_" + scan] = [os.path.join(test_sub["path"].iloc[i],scan,"Image-"+ str(img_centrale(test_sub["path"].iloc[i],scan)) +".dcm") for i in range(len(test_sub))]

Stock les images :

In [67]:
#Déclaration des listes d'images
test_sub_FLAIR = []
test_sub_T1w = []
test_sub_T1wCE = []
test_sub_T2w = []

#Alimentation des listes d'images
#FLAIR
for i in range(len(test_sub)) :
    test_sub_FLAIR.append(load_dicom(test_sub["path_img_FLAIR"].iloc[i]))
#T1w
for i in range(len(test_sub)) :
    test_sub_T1w.append(load_dicom(test_sub["path_img_T1w"].iloc[i]))
#T1wCE
for i in range(len(test_sub)) :
    test_sub_T1wCE.append(load_dicom(test_sub["path_img_T1wCE"].iloc[i]))
#T2w
for i in range(len(test_sub)) :
    test_sub_T2w.append(load_dicom(test_sub["path_img_T2w"].iloc[i]))
    
test_sub_FLAIR = np.array(test_sub_FLAIR)
test_sub_T1w = np.array(test_sub_T1w)
test_sub_T1wCE = np.array(test_sub_T1wCE)
test_sub_T2w = np.array(test_sub_T2w)

#Normaliser les images :
test_sub_FLAIR = test_sub_FLAIR.astype('float32')
test_sub_FLAIR /= 255

#Reshape : 
img_rows=test_sub_FLAIR[0].shape[0]
img_cols=test_sub_FLAIR[0].shape[1]
test_sub_FLAIR=test_sub_FLAIR.reshape(test_sub_FLAIR.shape[0],img_rows,img_cols,1)


Vérification avec le coupe FLAIR :

In [68]:
plt.imshow(test_sub_FLAIR[0])

# Création de la base convolutive

Source : https://www.tensorflow.org/tutorials/images/cnn

In [69]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
#32 : Nombre de noeuds, (3,3) -> Dimension du kernel (=Fenetre glissante)
#input_shape=(hauteur, largeur, canaux(RGB par exemple))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

#On visualise le modèle :
model.summary()

On remarque que les dimensions de largeur et de hauteur ont tendance à diminuer au fur et à mesure qu'on avance dans le réseau.
Pour layers.Conv2D(), le premier argument est le nombre de canaux en sortie (32 ou 64).

### Ajout de couches denses sur le dessus :

In [70]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(2)) #2 car il y a deux classes : MGMT = 1 et MGMT = 0

#On visualise le modèle :
model.summary()

# Compiler et entraîner le modèle avec la couche FLAIR :

In [71]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train2_FLAIR, MGMT, epochs=10,validation_data=(test2_FLAIR, MGMT_test))

## Evaluation du modèle

In [72]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test2_FLAIR, MGMT_test, verbose=2)

In [73]:
print(test_acc)

In [74]:
print(history.history.keys())

In [75]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.ylim([0.5, 1])
plt.legend(loc='lower right')

Précision de 56-57 %, en validation, pour ce modèle !

A faire : 
* Faire varier les paramètres du modèle
* Formater les images de la base test (compétition) pour prédire depuis un modèle
* Soumettre des prévisions 
* Trouver comment inclure les 4 types de coupes en entrée 
* Faire une fonction pour normaliser-array-reshape les images
* tester avec les images en noir et blanc (cmap="Greys")

## Remarques :

Pour le même échantillon et le même modèle, passer de 10 à 15 Epoch fait passer l'erreur en validation de 53,85 % à 54,40 %. Il y a donc une légère amélioration mais le temps de calcul est forcément plus long.

# Prédire et soumettre le modèle

In [76]:
MGMT_predict = model.predict(test_sub_FLAIR)
classes = np.argmax(MGMT_predict, axis = 1)
print(classes)

Ainsi, on obtient les prévisions sur l'échantillon test des images. Nous voulons maintenant soumettre ces prévisions : 

In [77]:
test_sub["MGMT_value"] = classes #ajout des prévisions à la base à soumettre
test_sub = test_sub[["BraTS21ID","MGMT_value"]]
test_sub.head(10)
test_sub.to_csv("submission.csv",index=False)  #Sauvegarde en CSV

## Sauvegarde des modèles :

Modèle 1 : echantillon train de 400 individus et test de 182.
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2)) #2 car il y a deux classes : MGMT = 1 et MGMT = 0


Type de coupe utilisé : FLAIR


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train2_FLAIR, MGMT, epochs=10,validation_data=(test2_FLAIR, MGMT_test))

Précision en validation : 0,58